# Introduction
[Chainer](http://chainer.org/) とはニューラルネットの実装を簡単にしたフレームワークです。

* 今回は機械翻訳にニューラルネットを適用してみました。

![](./pictures/Chainer.jpg)

* 今回は機械翻訳を行っていただきます。

機械翻訳は機械が言語を別の言語に翻訳するものです。

機械翻訳にはいくつか種類があるのでここでも紹介しておきます。

* IBMモデル
 * [moses](http://www.statmt.org/moses/)というオープンソースで使用できるメジャーな機械翻訳のモデルですが、難しすぎて理解できない人を続出させる機械翻訳の鬼門です
* ニューラル機械翻訳
 * 翻訳元単語の辞書ベクトルを潜在空間ベクトルに落とし込み、ニューラルネットで翻訳先言語を学習させる手法

以下では、このChainerを利用しデータを準備するところから実際に言語モデルを構築し学習・評価を行うまでの手順を解説します。

1. [各種ライブラリ導入](#各種ライブラリ導入) 
2. [機械翻訳のクラス](#機械翻訳のクラス) 
3. [クラスの初期設定](#クラスの初期設定)
4. [モデルの保存](#モデルの保存) 
5. [モデルの読み込み](#モデルの読み込み)
6. [初期設定とForWard処理(Encoding Decoding)](#初期設定とForWard処理(Encoding Decoding))
7. [モデルの学習と予測のメソッドを実装](#モデルの学習と予測のメソッドを実装)
8. [各値を設定](#各値を設定)
9. [モデルの学習](#モデルの学習)
10. [予測](#予測)
11. [実行](#実行)



## 1.各種ライブラリ導入

Chainerの言語処理では多数のライブラリを導入します。



In [ ]:
import sys
import math
import numpy as np

from chainer import functions, optimizers

import util.generators as gens
from util.functions import trace, fill_batch
from util.model_file import ModelFile
from util.vocabulary import Vocabulary

from util.chainer_cpu_wrapper import wrapper

`導入するライブラリの代表例は下記です。

* `numpy`: 行列計算などの複雑な計算を行なうライブラリ
* `chainer`: Chainerの導入
* `util`:今回の処理で必要なライブラリが入っています。


## 2.機械翻訳のクラス

下記を設定しています。
* ニューラルネットを用いて機械翻訳用のモデルを構成しています。
ややこしいので各構成の説明

全体構成

![](./pictures/chainer_machine_translation1.png)

Encoder部
1. 翻訳元言語のBag of Wordsを潜在ベクトル空間に写像
2. 潜在ベクトル空間の値を隠れ層に遷移
3. 隠れ層から赤くなっている隠れ層まで遷移

Decoder部
4. 赤くなっている隠れ層からDecoder部の隠れ層で値を受け取る
5. 隠れ層から値を潜在ベクトル空間に写像
6. 潜在ベクトル空間から翻訳先言語のBag of Wordsに写像
7. 翻訳先言語のBag of Wordsに写像した値を次の単語の隠れ層に入れる
8. 隠れ層から隠れ層への遷移



In [ ]:
class EncoderDecoderModelMakeModel:
    def __init__(self):
        pass

    def make_model(self):
        self.model = wrapper.make_model(
            # encoder
            w_xi = functions.EmbedID(len(self.src_vocab), self.n_embed),       #1
            w_ip = functions.Linear(self.n_embed, 4 * self.n_hidden),          #2
            w_pp = functions.Linear(self.n_hidden, 4 * self.n_hidden),         #3
            # decoder
            w_pq = functions.Linear(self.n_hidden, 4 * self.n_hidden),         #4
            w_qj = functions.Linear(self.n_hidden, self.n_embed),              #5
            w_jy = functions.Linear(self.n_embed, len(self.trg_vocab)),        #6
            w_yq = functions.EmbedID(len(self.trg_vocab), 4 * self.n_hidden),  #7
            w_qq = functions.Linear(self.n_hidden, 4 * self.n_hidden),         #8
        )

## 3.クラスの初期設定

クラスの初期設定を行っています。

* クラスの生成
* 翻訳元言語の登録
* 翻訳先言語の登録
* 潜在ベクトル数の登録
* 隠れ層の登録
* ニューラル翻訳モデルの生成


In [ ]:
class EncoderDecoderModelNew(EncoderDecoderModelMakeModel):
    def __init__(self):
        pass

    @staticmethod
    def new(src_vocab, trg_vocab, n_embed, n_hidden):
        self = EncoderDecoderModel()
        print(dir(self))
        self.src_vocab = src_vocab
        self.trg_vocab = trg_vocab
        self.n_embed = n_embed
        self.n_hidden = n_hidden
        self.make_model()
        return self

## 4.モデルの保存

モデルが出力する各値を保存します。

* 翻訳元言語の語彙数
* 翻訳先言語の語彙数
* 潜在空間ベクトル
* 隠れ層
* モデル
* モデルが保持する各値

In [ ]:
class EncoderDecoderModelSave(EncoderDecoderModelNew):
    def __init__(self):
        pass

    def save(self, filename):
        with ModelFile(filename, 'w') as fp:
            self.src_vocab.save(fp.get_file_pointer())
            self.trg_vocab.save(fp.get_file_pointer())
            fp.write(self.n_embed)
            fp.write(self.n_hidden)
            wrapper.begin_model_access(self.model)
            fp.write_embed(self.model.w_xi)
            fp.write_linear(self.model.w_ip)
            fp.write_linear(self.model.w_pp)
            fp.write_linear(self.model.w_pq)
            fp.write_linear(self.model.w_qj)
            fp.write_linear(self.model.w_jy)
            fp.write_embed(self.model.w_yq)
            fp.write_linear(self.model.w_qq)
            wrapper.end_model_access(self.model)

## 5.モデルの読み込み

モデルを読み込む際の値の定義です。

* 翻訳元言語の語彙数
* 翻訳先言語の語彙数
* 潜在空間ベクトル
* 隠れ層
* モデル
* モデルが保持する各値

In [ ]:
class EncoderDecoderModelLoad(EncoderDecoderModelSave):
    def __init__(self):
        pass
    
    @staticmethod
    def load(filename):
        self = EncoderDecoderModel()
        with ModelFile(filename) as fp:
            self.src_vocab = Vocabulary.load(fp.get_file_pointer())
            self.trg_vocab = Vocabulary.load(fp.get_file_pointer())
            self.n_embed = int(fp.read())
            self.n_hidden = int(fp.read())
            self.make_model()
            wrapper.begin_model_access(self.model)
            fp.read_embed(self.model.w_xi)
            fp.read_linear(self.model.w_ip)
            fp.read_linear(self.model.w_pp)
            fp.read_linear(self.model.w_pq)
            fp.read_linear(self.model.w_qj)
            fp.read_linear(self.model.w_jy)
            fp.read_embed(self.model.w_yq)
            fp.read_linear(self.model.w_qq)
            wrapper.end_model_access(self.model)
        return self

## 6.初期設定とForWard処理(Encoding Decoding)

最適化のための初期設定とForWard処理の記述です。

最適化のための初期設定

* 最適化手法の設定（AdaGrad)
* モデル設定

ForWard処理

* 初期設定

Encoding処理

* 入力の値を潜在ベクトル空間に写像
* 潜在ベクトル空間のデータを隠れ層用のデータに変換
* 隠れ層のデータをLSTMで渡す

Decoding処理

学習処理

* 隠れ層から潜在ベクトル空間への写像処理
* 潜在ベクトル空間からBag Of Wordsへ写像
* ミニバッチのサイズ分、翻訳先言語を保持
* 正解の翻訳結果と予測した結果を照らし合わせて損失を計算
* 翻訳結果を保持
* 翻訳結果を仮説候補として追加
* 翻訳結果を潜在ベクトル空間に写像した値と隠れ層から伝搬した値を合わして次の隠れ層に伝搬させていく

予測

* 生成制限以下の範囲で予測を行う
* </s>まで予測したら終了するようになっている
* 学習とは違うので単純に予測したデータをlstmに加えている
* それ以外の処理は損失計算部分がない点以外同じ

In [ ]:
class EncoderDecoderModelInitForward(EncoderDecoderModelLoad):
    def __init__(self):
        pass

    def init_optimizer(self):
        self.opt = optimizers.AdaGrad(lr=0.01)
        self.opt.setup(self.model.collect_parameters())

    def forward(self, is_training, src_batch, trg_batch = None, generation_limit = None):
        m = self.model
        tanh = functions.tanh
        lstm = functions.lstm
        batch_size = len(src_batch)
        src_len = len(src_batch[0])
        src_stoi = self.src_vocab.stoi
        trg_stoi = self.trg_vocab.stoi
        trg_itos = self.trg_vocab.itos
        s_c = wrapper.zeros((batch_size, self.n_hidden))
        
        # encoding
        s_x = wrapper.make_var([src_stoi('</s>') for _ in range(batch_size)], dtype=np.int32)
        s_i = tanh(m.w_xi(s_x))
        s_c, s_p = lstm(s_c, m.w_ip(s_i))

        for l in reversed(range(src_len)):
            s_x = wrapper.make_var([src_stoi(src_batch[k][l]) for k in range(batch_size)], dtype=np.int32)
            s_i = tanh(m.w_xi(s_x))
            s_c, s_p = lstm(s_c, m.w_ip(s_i) + m.w_pp(s_p))

        s_c, s_q = lstm(s_c, m.w_pq(s_p))
        hyp_batch = [[] for _ in range(batch_size)]
        
        # decoding
        if is_training:
            accum_loss = wrapper.zeros(())
            trg_len = len(trg_batch[0])
            
            for l in range(trg_len):
                s_j = tanh(m.w_qj(s_q))
                r_y = m.w_jy(s_j)
                s_t = wrapper.make_var([trg_stoi(trg_batch[k][l]) for k in range(batch_size)], dtype=np.int32)
                accum_loss += functions.softmax_cross_entropy(r_y, s_t)
                output = wrapper.get_data(r_y).argmax(1)

                for k in range(batch_size):
                    hyp_batch[k].append(trg_itos(output[k]))

                s_c, s_q = lstm(s_c, m.w_yq(s_t) + m.w_qq(s_q))

            return hyp_batch, accum_loss
        else:
            while len(hyp_batch[0]) < generation_limit:
                s_j = tanh(m.w_qj(s_q))
                r_y = m.w_jy(s_j)
                output = wrapper.get_data(r_y).argmax(1)

                for k in range(batch_size):
                    hyp_batch[k].append(trg_itos(output[k]))

                if all(hyp_batch[k][-1] == '</s>' for k in range(batch_size)): break

                s_y = wrapper.make_var(output, dtype=np.int32)
                s_c, s_q = lstm(s_c, m.w_yq(s_y) + m.w_qq(s_q))
            
            return hyp_batch

## 7.モデルの学習と予測のメソッドを実装

モデルの学習と予測処理の実装です。

学習

* 初期化
* Forward処理で仮説候補と損失を算出
* backward処理で損失を計算
* 最適化における発散の防止
* 値を最適化

予測

* Forward処理に記述した予測処理を行います

In [ ]:
class EncoderDecoderModel(EncoderDecoderModelInitForward):
    def __init__(self):
        pass
    
    def train(self, src_batch, trg_batch):
        self.opt.zero_grads()
        hyp_batch, accum_loss = self.forward(True, src_batch, trg_batch=trg_batch)
        accum_loss.backward()
        self.opt.clip_grads(10)
        self.opt.update()
        return hyp_batch

    def predict(self, src_batch, generation_limit):
        return self.forward(False, src_batch, generation_limit=generation_limit)

## 8.各値を設定

各値を設定

* モードを学習かテストか設定
* 翻訳元言語の設定
* 翻訳先言語の設定
* 語彙の設定
* 潜在空間の設定
* 隠れ層の設定
* 学習回数の設定
* ミニバッチサイズの設定
* 最大予測言語数の設定


In [ ]:
mode = "train"
source = "source.txt"
target = "target.txt"
vocab = 32768
embed = 256
hidden = 512
epoch = 100
minibatch = 64
generation_limit = 256

## 9.モデルの学習

学習用のメソッド

* 翻訳元言語を処理用の変数に変換
* 翻訳先言語を処理用の変数に変換
* 学習用のモデル設定

学習回数分、下記の処理を行う

* 翻訳元言語をlist化
* 翻訳先言語をlist化
* sorted_parellel処理はややこしいので少し解説
　翻訳元言語と翻訳先言語のリストを100×ミニバッチのサイズ分渡すとタプル形式でソートして返してくれます。
　それをbatch関数でミニバッチのサイズ分取得しているのがgen3の処理です。
 
* 初期化
* gen3を用いて翻訳元言語と翻訳先言語を取り出し
　すべての文字列の末尾に"</s>"を挿入する

* 仮説候補を取得
* 翻訳元言語、翻訳先言語、翻訳仮説を表示
* 各学習ごとにモデルを保存

In [ ]:
def train_model():
    trace('making vocaburaries ...')
    src_vocab = Vocabulary.new(gens.word_list(source), vocab)
    trg_vocab = Vocabulary.new(gens.word_list(target), vocab)

    trace('making model ...')
    model = EncoderDecoderModel.new(src_vocab, trg_vocab, embed, hidden)

    for i_epoch in range(epoch):
        trace('epoch %d/%d: ' % (i_epoch + 1, epoch))
        trained = 0
        gen1 = gens.word_list(source)
        gen2 = gens.word_list(target)
        gen3 = gens.batch(gens.sorted_parallel(gen1, gen2, 100 * minibatch), minibatch)
        model.init_optimizer()

        for src_batch, trg_batch in gen3:
            src_batch = fill_batch(src_batch)
            trg_batch = fill_batch(trg_batch)
            K = len(src_batch)
            hyp_batch = model.train(src_batch, trg_batch)

            for k in range(K):
                trace('epoch %3d/%3d, sample %8d' % (i_epoch + 1, epoch, trained + k + 1))
                trace('  src = ' + ' '.join([x if x != '</s>' else '*' for x in src_batch[k]]))
                trace('  trg = ' + ' '.join([x if x != '</s>' else '*' for x in trg_batch[k]]))
                trace('  hyp = ' + ' '.join([x if x != '</s>' else '*' for x in hyp_batch[k]]))

            trained += K

        trace('saving model ...')
        model.save(model + '.%03d' % (epoch + 1))

    trace('finished.')

## 10.予測

予測

* 学習したモデルを読み込む
* 翻訳元言語をミニバッチのサイズ分読み込んで、仮説候補をモデルから予測
* 仮説候補を表示


In [ ]:
def test_model(args):
    trace('loading model ...')
    model = EncoderDecoderModel.load(model)
    
    trace('generating translation ...')
    generated = 0

    with open(target, 'w') as fp:
        for src_batch in gens.batch(gens.word_list(source), minibatch):
            src_batch = fill_batch(src_batch)
            K = len(src_batch)

            trace('sample %8d - %8d ...' % (generated + 1, generated + K))
            hyp_batch = model.predict(src_batch, generation_limit)

            for hyp in hyp_batch:
                hyp.append('</s>')
                hyp = hyp[:hyp.index('</s>')]
                print(' '.join(hyp), file=fp)

            generated += K

    trace('finished.')

## 11.実行

In [ ]:
def main():

    trace('initializing ...')
    wrapper.init()

    if mode == 'train': train_model()
    elif mode == 'test': test_model()


if __name__ == '__main__':
    main()